# DB2-Salesforce connector: Basic user information updates

In [1]:
# Parameters
hours_range = 24*365*30 # number of hours to look back

# API settings
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

In [2]:
# Parameters
import sqlalchemy as sql
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce


nanoHUB - Serving Students, Researchers & Instructors
Obtained Salesforce access token ...... True


In [3]:
from nanoHUB.logger import logger
log = logger('task_user_basic_updates')

In [4]:
import sys
import pandas as pd
import datetime

In [5]:
def get_df_for_yesterday(df: pd.DataFrame, column_name: str = 'registerDate'): 
    df[column_name] = pd.to_datetime(df[column_name])
    todays_date = df[column_name].dt.date.max() - datetime.timedelta(days=1)
    return get_df_for(todays_date, df, column_name)

def get_df_for(for_date: datetime, df: pd.DataFrame, column_name: str = 'registerDate'):
    df[column_name] = pd.to_datetime(df[column_name])
    return df[df[column_name].dt.date == for_date]

def get_number_of_registered_users_for(for_date: datetime, df: pd.DataFrame, column_name: str = 'registerDate'):
    df = get_df_for(for_date, df, column_name)
    log.info("Number of users who registered on (%s) is: %d" %(for_date, len(df.index)))
    return len(df.index)
    
def get_number_of_registered_users_for_yesterday(df: pd.DataFrame, column_name: str = 'registerDate'):
    df[column_name] = pd.to_datetime(df[column_name])
    todays_date = df[column_name].dt.date.max() - datetime.timedelta(days=1)
    return get_number_of_registered_users_for(todays_date, df, column_name)

## Obtain contacts from DB2
that have last visit date within range of interest specified by day_range

In [6]:
# Hourly update for new registrations
date_cutoff = (datetime.datetime.today().date() - datetime.timedelta(hours=hours_range)).strftime('%Y-%m-%d')
display(date_cutoff)

'1991-12-10'

In [7]:
sql_query = r"""
SELECT user_id
     , username
     , registerDate
     , lastvisitDate
     , raw_full_name
     , title
     , first_name
     , middle_name
     , last_name
     , LOWER(MAX(calc_data.organization)) AS organization
     , LOWER(MAX(calc_data.orgtype))      AS orgtype
     , LOWER(MAX(calc_data.orcid))        AS orcid
     , email
     , LOWER(RIGHT(email, length(email)-INSTR(email, '@'))) AS email_host
     , block
     , sendEmail
FROM (
       SELECT user_info.*
            , CASE
                WHEN 0 = LOCATE(' ', user_info.REST_OF_NAME)
                  THEN NULL
                ELSE SUBSTRING(
                        user_info.REST_OF_NAME
                  , 1
                  , LOCATE(' ', user_info.REST_OF_NAME) - 1
                  )
         END AS                                                                                 middle_name
            , SUBSTRING(
               user_info.REST_OF_NAME
         , 1 + LOCATE(' ', user_info.REST_OF_NAME)
         , LENGTH(user_info.REST_OF_NAME)
         )   AS                                                                                 last_name
            , (CASE WHEN profiles.profile_key = 'organization' THEN profiles.profile_value end) organization
            , (CASE WHEN profiles.profile_key = 'orcid' THEN profiles.profile_value end)        orcid
            , (CASE WHEN profiles.profile_key = 'orgtype' THEN profiles.profile_value end)      orgtype
  
       FROM (
              SELECT user_id,
                     username,
                     block,
                     email,
                     sendEmail,
                     registerDate,
                     lastvisitDate
                      ,
                     title.title
                      ,
                     CASE
                       WHEN 0 = LOCATE(' ', title.REST_OF_NAME)
                         THEN title.REST_OF_NAME
                       ELSE SUBSTRING(
                               title.REST_OF_NAME
                         , 1
                         , LOCATE(' ', title.REST_OF_NAME) - 1
                         )
                       END AS first_name
                      ,
                     CASE
                       WHEN 0 = LOCATE(' ', title.REST_OF_NAME)
                         THEN NULL
                       ELSE SUBSTRING(
                               title.REST_OF_NAME
                         , LOCATE(' ', title.REST_OF_NAME) + 1
                         , LENGTH(title.REST_OF_NAME)
                         )
                       END AS REST_OF_NAME
                      ,
                     title.raw_full_name
              FROM (
                     SELECT user_id,
                            username,
                            block,
                            email,
                            sendEmail,
                            registerDate,
                            lastvisitDate
                             ,
                            CASE
                              WHEN SUBSTRING(TEST_DATA.full_name, 1, 3) IN ('PROF ', 'MR ', 'MS ', 'DR ', 'MRS ')
                                THEN LTRIM(RTRIM(SUBSTRING(TEST_DATA.full_name, 1, 3)))
                              ELSE NULL
                              END AS title
                             ,
                            CASE
                              WHEN SUBSTRING(TEST_DATA.full_name, 1, 3) IN ('PROF ', 'MR ', 'MS ', 'DR ', 'MRS ')
                                THEN LTRIM(RTRIM(SUBSTRING(TEST_DATA.full_name, 4, LENGTH(TEST_DATA.full_name))))
                              ELSE LTRIM(RTRIM(TEST_DATA.full_name))
                              END AS REST_OF_NAME
                             ,
                            TEST_DATA.raw_full_name
                     FROM (
                            SELECT REPLACE(REPLACE(LTRIM(RTRIM(name)), '  ', ' '), '  ', ' ') AS full_name,
                                   name                                                       AS raw_full_name,
                                   user_id,
                                   username,
                                   block,
                                   email,
                                   sendEmail,
                                   registerDate,
                                   lastvisitDate
                            FROM (
                                   SELECT id AS user_id,
                                          name,
                                          username,
                                          block,
                                          email,
                                          sendEmail,
                                          registerDate,
                                          lastvisitDate
                                   FROM nanohub.jos_users
                                   WHERE registerDate >= '%s'
                                 ) RAW_DATA
                          ) TEST_DATA
                   ) title
            ) user_info
              LEFT JOIN nanohub.jos_user_profiles profiles
                        ON profiles.user_id = user_info.user_id
                          AND profiles.profile_key IN ('orgtype', 'organization', 'orcid')
     ) AS calc_data
GROUP BY calc_data.user_id
;
"""

sql_query = sql_query % date_cutoff 

In [8]:
df = pd.read_sql_query(sql_query, nanohub_db, parse_dates=['registerDate', 'lastvisitDate'])
# display
display(df.head())
display(df.tail())

,user_id,username,registerDate,lastvisitDate,raw_full_name,title,first_name,middle_name,last_name,organization,orgtype,orcid,email,email_host,block,sendEmail
0,998,hubrepo,2014-11-13 21:09:09,NaT,hubrepo hubrepo,None,hubrepo,None,hubrepo,None,None,None,nkissebe@gmail.com,gmail.com,0,0
1,1683,support,2008-11-19 22:51:04,2008-11-19 23:55:30,nanoHUB support,None,nanoHUB,None,support,purdue university,university,None,support@nanohub.org,nanohub.org,0,0
2,1684,gridstat,2008-11-18 17:29:56,2020-02-14 18:50:14,Grid Statistics,None,Grid,None,Statistics,purdue university,universitystaff,None,gridstat@nanohub.org,nanohub.org,0,0
3,1685,ncn,2008-11-11 19:17:04,NaT,NCN NCN,None,NCN,None,NCN,purdue university,None,None,ncn@nanohub.org,nanohub.org,0,0
4,1686,nanohub,2014-06-26 19:38:57,NaT,nanoHUB nanoHUB,None,nanoHUB,None,nanoHUB,purdue university,None,None,apps@nanohub.org,nanohub.org,0,0


,user_id,username,registerDate,lastvisitDate,raw_full_name,title,first_name,middle_name,last_name,organization,orgtype,orcid,email,email_host,block,sendEmail
265572,347883,-186860,2021-12-02 11:51:31,2021-12-02 11:56:27,Arun Jangir,None,Arun,None,Jangir,None,None,None,-186860@invalid,invalid,0,-1
265573,347885,-186862,2021-12-02 11:55:20,2021-12-02 11:56:27,Anthony Walton,None,Anthony,None,Walton,None,None,None,-186862@invalid,invalid,0,-1
265574,347886,sourav.sm423,2021-12-02 12:15:54,2021-12-02 12:15:55,Sourav Mondal,None,Sourav,None,Mondal,maulana abul kalam azad university of technology,universitygraduate,None,sourav.sm423@gmail.com,gmail.com,0,0
265575,347888,buyambienonlineovernight,2021-12-02 12:26:34,2021-12-02 12:27:33,Buy Ambien Online Overnight,None,Buy,Ambien,Online Overnight,"university of california, los angeles",universitygraduate,None,lindseydavid5236@gmail.com,gmail.com,0,0
265576,347889,-186865,2021-12-02 12:31:55,2021-12-02 12:31:55,chris wallium,None,chris,None,wallium,None,None,None,-186865@invalid,invalid,0,-1


In [9]:
display(get_number_of_registered_users_for_yesterday(df))

[INFO] [1175540541 - task_user_basic_updates]: Number of users who registered on (2021-12-01) is: 73 [1175540541.get_number_of_registered_users_for:12]


73

In [10]:
# get domain by nanoHUB organization field
df['organization'] = df.organization.astype('str').apply(str.lower)

df.loc[(df['organization']=='nan'),'organization'] = None

In [11]:
display(df.head())

,user_id,username,registerDate,lastvisitDate,raw_full_name,title,first_name,middle_name,last_name,organization,orgtype,orcid,email,email_host,block,sendEmail
0,998,hubrepo,2014-11-13 21:09:09,NaT,hubrepo hubrepo,None,hubrepo,None,hubrepo,none,None,None,nkissebe@gmail.com,gmail.com,0,0
1,1683,support,2008-11-19 22:51:04,2008-11-19 23:55:30,nanoHUB support,None,nanoHUB,None,support,purdue university,university,None,support@nanohub.org,nanohub.org,0,0
2,1684,gridstat,2008-11-18 17:29:56,2020-02-14 18:50:14,Grid Statistics,None,Grid,None,Statistics,purdue university,universitystaff,None,gridstat@nanohub.org,nanohub.org,0,0
3,1685,ncn,2008-11-11 19:17:04,NaT,NCN NCN,None,NCN,None,NCN,purdue university,None,None,ncn@nanohub.org,nanohub.org,0,0
4,1686,nanohub,2014-06-26 19:38:57,NaT,nanoHUB nanoHUB,None,nanoHUB,None,nanoHUB,purdue university,None,None,apps@nanohub.org,nanohub.org,0,0


In [12]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
s_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/saxenap/venv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import re

def clean_domain(this_domain_name):

    if not this_domain_name:
        return None

    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z]+", " ", this_domain_name.lower())
    name_list = this_domain_name.split(' ')

    # remove all stop words
    name_list = ['' if x in s_words else x for x in name_list]

    return set(filter(None, name_list))

In [14]:
# clean nanohub org name
df['org_cleaned_set'] = df.organization.apply(clean_domain)

# hash cleaned clean nanohub org name
df['org_cleaned_hash'] = df['org_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)


## Collaborative filtering for commerical email hosts

In [15]:
def predict_if_commerical(this_email_host_df):
    # predict if this email host is commerical
    top_cnt = this_email_host_df['org_cleaned_hash'].value_counts()

    if top_cnt.shape[0] > 0:
        if (top_cnt.iloc[0]/top_cnt.sum()) < 0.2:
            return True
        else:
            return False

    else:
        return False

is_email_commerical = df[['email_host', 'org_cleaned_hash']].groupby('email_host').apply(predict_if_commerical)

In [16]:
# mark commerical email addresses
df = pd.merge(df, is_email_commerical.reset_index(name='is_email_commerical'), how='left', left_on='email_host', right_on='email_host' ,)

In [17]:
display(get_number_of_registered_users_for_yesterday(df))

[INFO] [1175540541 - task_user_basic_updates]: Number of users who registered on (2021-12-01) is: 73 [1175540541.get_number_of_registered_users_for:12]


73

In [18]:
wang159_myrmekes_db = application.new_db_engine('wang159_myrmekes')

sql_query = """
SELECT 
    name AS domain_name, 
    domain, 
    industry, 
    `size range` as size, 
    country 
FROM wang159_myrmekes.companies_email_domain
;
"""
domain_df = pd.read_sql_query(sql_query, wang159_myrmekes_db)
display(domain_df.head())

,domain_name,domain,industry,size,country
0,strategic interns,strategicinterns.com,education management,1-10,india
1,bosque y comunidad,bosqueycomunidad.org,non-profit organization management,1-10,spain
2,alaric as,alaric.no,management consulting,1-10,None
3,tws technology inc.,talkwithsam.co,computer software,1-10,None
4,"cardinal strategies, llc",cardinal-strategies.com,civil engineering,11-50,united states


In [19]:
# make sure domain is unique and drop NaN
domain_df = domain_df.drop_duplicates(subset='domain', keep='first').dropna()

# clean domain name
domain_df['domain_cleaned_set'] = domain_df.domain_name.apply(clean_domain)

# hash cleaned domain name
domain_df['domain_cleaned_hash'] = domain_df['domain_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

In [20]:
# obtain domain estimation from email addresses
df = pd.merge(df, domain_df[~domain_df.domain.isin(is_email_commerical[is_email_commerical].index)]['domain'], how='left', left_on='email_host', right_on='domain').rename(columns={'domain':'domain_by_email'})

# display
display(df.head())
display(df.tail())

,user_id,username,registerDate,lastvisitDate,raw_full_name,title,first_name,middle_name,last_name,organization,orgtype,orcid,email,email_host,block,sendEmail,org_cleaned_set,org_cleaned_hash,is_email_commerical,domain_by_email
0,998,hubrepo,2014-11-13 21:09:09,NaT,hubrepo hubrepo,None,hubrepo,None,hubrepo,none,None,None,nkissebe@gmail.com,gmail.com,0,0,{none},none,False,NaN
1,1683,support,2008-11-19 22:51:04,2008-11-19 23:55:30,nanoHUB support,None,nanoHUB,None,support,purdue university,university,None,support@nanohub.org,nanohub.org,0,0,"{purdue, university}",purdue-university,False,NaN
2,1684,gridstat,2008-11-18 17:29:56,2020-02-14 18:50:14,Grid Statistics,None,Grid,None,Statistics,purdue university,universitystaff,None,gridstat@nanohub.org,nanohub.org,0,0,"{purdue, university}",purdue-university,False,NaN
3,1685,ncn,2008-11-11 19:17:04,NaT,NCN NCN,None,NCN,None,NCN,purdue university,None,None,ncn@nanohub.org,nanohub.org,0,0,"{purdue, university}",purdue-university,False,NaN
4,1686,nanohub,2014-06-26 19:38:57,NaT,nanoHUB nanoHUB,None,nanoHUB,None,nanoHUB,purdue university,None,None,apps@nanohub.org,nanohub.org,0,0,"{purdue, university}",purdue-university,False,NaN


,user_id,username,registerDate,lastvisitDate,raw_full_name,title,first_name,middle_name,last_name,organization,orgtype,orcid,email,email_host,block,sendEmail,org_cleaned_set,org_cleaned_hash,is_email_commerical,domain_by_email
265572,347883,-186860,2021-12-02 11:51:31,2021-12-02 11:56:27,Arun Jangir,None,Arun,None,Jangir,none,None,None,-186860@invalid,invalid,0,-1,{none},none,False,NaN
265573,347885,-186862,2021-12-02 11:55:20,2021-12-02 11:56:27,Anthony Walton,None,Anthony,None,Walton,none,None,None,-186862@invalid,invalid,0,-1,{none},none,False,NaN
265574,347886,sourav.sm423,2021-12-02 12:15:54,2021-12-02 12:15:55,Sourav Mondal,None,Sourav,None,Mondal,maulana abul kalam azad university of technology,universitygraduate,None,sourav.sm423@gmail.com,gmail.com,0,0,"{maulana, azad, abul, kalam, technology, unive...",abul-azad-kalam-maulana-technology-university,False,NaN
265575,347888,buyambienonlineovernight,2021-12-02 12:26:34,2021-12-02 12:27:33,Buy Ambien Online Overnight,None,Buy,Ambien,Online Overnight,"university of california, los angeles",universitygraduate,None,lindseydavid5236@gmail.com,gmail.com,0,0,"{los, university, california, angeles}",angeles-california-los-university,False,NaN
265576,347889,-186865,2021-12-02 12:31:55,2021-12-02 12:31:55,chris wallium,None,chris,None,wallium,none,None,None,-186865@invalid,invalid,0,-1,{none},none,False,NaN


## Fuzzy derive organization from nanoHUB profiles

In [21]:
# attempt direct join by hash
domain_subset_df = domain_df[domain_df.domain.isin(df.email_host.unique())][['domain', 'domain_cleaned_hash']]
domain_subset_df = domain_subset_df[~domain_subset_df.domain.isna()&~domain_subset_df.domain_cleaned_hash.isna()]

derived_df = pd.merge(df, domain_subset_df.rename(columns={'domain':'domain_by_profile'}),how='left', left_on='org_cleaned_hash', right_on='domain_cleaned_hash').drop('domain_cleaned_hash', axis=1)

# display
display(derived_df.sample(5))

,user_id,username,registerDate,lastvisitDate,raw_full_name,title,first_name,middle_name,last_name,organization,...,orcid,email,email_host,block,sendEmail,org_cleaned_set,org_cleaned_hash,is_email_commerical,domain_by_email,domain_by_profile
213750,282742,brooksidemachining,2020-04-01 14:37:05,2020-04-01 15:02:55,Brook Side Side Machining,None,Brook,Side,Side Machining,none,...,None,brooksidemachining1@gmail.com,gmail.com,1,1,{none},none,False,NaN,NaN
257338,335929,dallaman,2021-08-23 21:24:37,2021-12-01 03:19:42,David Michael Allaman,None,David,Michael,Allaman,purdue university,...,None,dallaman@purdue.edu,purdue.edu,0,0,"{purdue, university}",purdue-university,False,purdue.edu,purdue.edu
69071,72418,khuili,2012-11-03 16:54:44,2013-07-25 16:44:16,khuili med,None,khuili,None,med,faculte des sciences et technique benimellal,...,None,khuili01@hotmail.com,hotmail.com,0,0,"{et, des, technique, sciences, benimellal, fac...",benimellal-des-et-faculte-sciences-technique,False,NaN,NaN
141510,166089,arya.raychaudhuri,2017-03-07 04:45:48,2017-04-27 17:38:17,Arya Raychaudhuri,None,Arya,None,Raychaudhuri,none,...,None,arya.raychaudhuri@gmail.com,gmail.com,0,0,{none},none,False,NaN,NaN
203038,268055,milankshah135,2019-10-09 02:35:54,2019-12-04 21:53:44,Milan Shah,None,Milan,None,Shah,none,...,None,milankshah135@gmail.com,gmail.com,0,-1,{none},none,False,NaN,NaN


In [22]:
# attempt to reduce domain_df size to speedup Jaccard calculation
from itertools import chain

all_word_set = set(chain.from_iterable(derived_df[(derived_df['domain_by_profile'].isna() & (~derived_df['organization'].isna()))].org_cleaned_set.values))

# select the domain_df entry with words from all_word_set
domain_subset_df = domain_df[domain_df.domain_cleaned_set.notnull()]
domain_subset_df = domain_subset_df[domain_subset_df.domain_cleaned_set.apply(lambda x: True if len(x.intersection(all_word_set)) > 0 else False)]

In [23]:
def get_org(this_df):
    # look at the cohort with this label, do they have a commonly agreed domain_by_email? If so, use it
    this_dbe_count = this_df.domain_by_email.value_counts()

    if this_dbe_count.shape[0] < 1:
        return None

    this_top_dbe_perc = this_dbe_count.iloc[0]/this_dbe_count.sum()

    # if top choice occupies majority of cases, then this cohort can be implied to this domain_by_email address
    if this_top_dbe_perc > 0.5:
        return this_dbe_count.index[0]


derived_hash = derived_df.groupby('org_cleaned_hash').apply(get_org)

In [24]:
derived_df = pd.merge(derived_df, derived_hash.reset_index(name='domain_by_profile_infer'), how='left', left_on='org_cleaned_hash', right_on='org_cleaned_hash')

In [25]:
# calculate a composite domain based on domain_by_email, domain_by_profile, and domain_by_profile_infer
def get_composite_domain(this_df):

    # if domain found via profile with direct match, use it
    if this_df.domain_by_profile != '':
        return this_df.domain_by_profile

    # if domain found via email, use it
    if this_df.domain_by_email != '':
        return this_df.domain_by_email

        # if domain found via profile but not with a direct match, use it
    if this_df.domain_by_profile_infer != '':
        return this_df.domain_by_profile_infer

    # else, no organization can be concluded, return none
    return None


derived_df['domain_final'] = derived_df[['domain_by_email', 'domain_by_profile', 'domain_by_profile_infer']].fillna('').apply(get_composite_domain, axis=1)

In [26]:
derived_df[derived_df.email_host == 'yahoo.com'][['domain_final', 'domain_by_email']]

,domain_final,domain_by_email
8,yahoo.com,yahoo.com
17,yahoo.com,yahoo.com
42,yahoo.com,yahoo.com
52,yahoo.com,yahoo.com
76,yahoo.com,yahoo.com
...,...,...
266651,yahoo.com,yahoo.com
266681,yahoo.com,yahoo.com
266687,csu.edu,yahoo.com
266767,yahoo.com,yahoo.com


## Update Salesforce institution

In [27]:
# get a unique list of organizations that appears in contact
#org_df = derived_df[['domain_name', 'domain', 'industry', 'size', 'country']].drop_duplicates().dropna()
org_df = pd.merge(derived_df[['domain_final']].drop_duplicates().dropna(), domain_df, how='inner', left_on='domain_final', right_on='domain')
org_df = org_df[['domain_name', 'domain', 'industry', 'size', 'country']]

# display
display(org_df.head(2))

,domain_name,domain,industry,size,country
0,purdue university,purdue.edu,higher education,10001+,united states
1,yahoo,yahoo.com,internet,10001+,united states


In [28]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name'] = org_df['domain_name']

df_sf['Size_Range__c'] = org_df['size'].fillna('Unknown')
df_sf['Country__c'] = org_df['country']
df_sf['Domain__c'] = org_df['domain']
df_sf['Industry__c'] = org_df['industry']

sf_original_fields = df_sf.columns

# display
display(df_sf.head())
display(df_sf.tail())

,Name,Size_Range__c,Country__c,Domain__c,Industry__c
0,purdue university,10001+,united states,purdue.edu,higher education
1,yahoo,10001+,united states,yahoo.com,internet
2,netzero inc,1-10,united states,netzero.net,computer software
3,georgetown university,5001-10000,united states,georgetown.edu,higher education
4,university of cincinnati,10001+,united states,uc.edu,higher education


,Name,Size_Range__c,Country__c,Domain__c,Industry__c
3712,loeb & loeb llp,501-1000,united states,loeb.com,law practice
3713,st. petersburg college,1001-5000,united states,spcollege.edu,higher education
3714,canadian nuclear laboratories,1001-5000,canada,cnl.ca,research
3715,noon,501-1000,united arab emirates,noon.com,internet
3716,skvare,1-10,united states,skvare.com,information technology and services


In [29]:


db_s.object_id = 'organization__c'
db_s.external_id = 'Domain__c'

In [30]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000bmmmgAAA
hello
[Success] CSV upload successful. Job ID = 7505w00000bmmmgAAA
[Success] Closing job successful. Job ID = 7505w00000bmmmgAAA


In [31]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000bmmmgAAA',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-12-02T15:11:20.000+0000',
 'systemModstamp': '2021-12-02T15:11:21.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Domain__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [32]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


## Obtain newly updated organization IDs from Salesforce

In [33]:
# create DB2 to Salesforce API object


In [34]:
# get Salesforce ID for organizations
sf_org_ID_df = db_s.query_data('SELECT Id, Domain__c FROM organization__c')

# display
sf_org_ID_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000bmmmlAAA
{"id":"7505w00000bmmmlAAA","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-12-02T15:11:22.000+0000","systemModstamp":"2021-12-02T15:11:22.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000bmmmlAAA","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-12-02T15:11:22.000+0000","systemModstamp":"2021-12-02T15:11:23.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000bmmmlAAA","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDa

,Domain__c,Id
0,upenn.edu,a0r5w00000V42c0AAB
1,fer.hr,a0r5w00000V42c1AAB
2,wheatoncollege.edu,a0r5w00000V42c2AAB


In [35]:
# join salesforce ID back to contact DF
df = pd.merge(sf_org_ID_df, derived_df, how='right', left_on='Domain__c', right_on='domain_final').drop('Domain__c', axis=1).rename(columns={'Id':'Salesforce_org_ID'})

In [36]:
# display
display(df.head(2))

,Salesforce_org_ID,user_id,username,registerDate,lastvisitDate,raw_full_name,title,first_name,middle_name,last_name,...,email_host,block,sendEmail,org_cleaned_set,org_cleaned_hash,is_email_commerical,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,NaN,998,hubrepo,2014-11-13 21:09:09,NaT,hubrepo hubrepo,None,hubrepo,None,hubrepo,...,gmail.com,0,0,{none},none,False,NaN,NaN,None,None
1,a0r5w00000V42cCAAR,1683,support,2008-11-19 22:51:04,2008-11-19 23:55:30,nanoHUB support,None,nanoHUB,None,support,...,nanohub.org,0,0,"{purdue, university}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu


## Match data with Salesforce format

In [37]:
display(get_number_of_registered_users_for_yesterday(df))

[INFO] [1175540541 - task_user_basic_updates]: Number of users who registered on (2021-12-01) is: 73 [1175540541.get_number_of_registered_users_for:12]


73

In [38]:
# Make sure NaN and NaT values are taken care of here
df['raw_full_name'] = df.raw_full_name.str.strip()

display(df['raw_full_name'].notnull())

0         True
1         True
2         True
3         True
4         True
          ... 
267057    True
267058    True
267059    True
267060    True
267061    True
Name: raw_full_name, Length: 267062, dtype: bool

In [39]:
display(get_number_of_registered_users_for_yesterday(df))

[INFO] [1175540541 - task_user_basic_updates]: Number of users who registered on (2021-12-01) is: 73 [1175540541.get_number_of_registered_users_for:12]


73

In [40]:
df = df[df['raw_full_name'].notnull()]

display(df.head())
display(df.tail())

,Salesforce_org_ID,user_id,username,registerDate,lastvisitDate,raw_full_name,title,first_name,middle_name,last_name,...,email_host,block,sendEmail,org_cleaned_set,org_cleaned_hash,is_email_commerical,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,NaN,998,hubrepo,2014-11-13 21:09:09,NaT,hubrepo hubrepo,None,hubrepo,None,hubrepo,...,gmail.com,0,0,{none},none,False,NaN,NaN,None,None
1,a0r5w00000V42cCAAR,1683,support,2008-11-19 22:51:04,2008-11-19 23:55:30,nanoHUB support,None,nanoHUB,None,support,...,nanohub.org,0,0,"{purdue, university}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu
2,a0r5w00000V42cCAAR,1684,gridstat,2008-11-18 17:29:56,2020-02-14 18:50:14,Grid Statistics,None,Grid,None,Statistics,...,nanohub.org,0,0,"{purdue, university}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu
3,a0r5w00000V42cCAAR,1685,ncn,2008-11-11 19:17:04,NaT,NCN NCN,None,NCN,None,NCN,...,nanohub.org,0,0,"{purdue, university}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu
4,a0r5w00000V42cCAAR,1686,nanohub,2014-06-26 19:38:57,NaT,nanoHUB nanoHUB,None,nanoHUB,None,nanoHUB,...,nanohub.org,0,0,"{purdue, university}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu


,Salesforce_org_ID,user_id,username,registerDate,lastvisitDate,raw_full_name,title,first_name,middle_name,last_name,...,email_host,block,sendEmail,org_cleaned_set,org_cleaned_hash,is_email_commerical,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
267057,NaN,347883,-186860,2021-12-02 11:51:31,2021-12-02 11:56:27,Arun Jangir,None,Arun,None,Jangir,...,invalid,0,-1,{none},none,False,NaN,NaN,None,None
267058,NaN,347885,-186862,2021-12-02 11:55:20,2021-12-02 11:56:27,Anthony Walton,None,Anthony,None,Walton,...,invalid,0,-1,{none},none,False,NaN,NaN,None,None
267059,NaN,347886,sourav.sm423,2021-12-02 12:15:54,2021-12-02 12:15:55,Sourav Mondal,None,Sourav,None,Mondal,...,gmail.com,0,0,"{maulana, azad, abul, kalam, technology, unive...",abul-azad-kalam-maulana-technology-university,False,NaN,NaN,None,None
267060,a0r5w00000V42okAAB,347888,buyambienonlineovernight,2021-12-02 12:26:34,2021-12-02 12:27:33,Buy Ambien Online Overnight,None,Buy,Ambien,Online Overnight,...,gmail.com,0,0,"{los, university, california, angeles}",angeles-california-los-university,False,NaN,NaN,ucla.edu,ucla.edu
267061,NaN,347889,-186865,2021-12-02 12:31:55,2021-12-02 12:31:55,chris wallium,None,chris,None,wallium,...,invalid,0,-1,{none},none,False,NaN,NaN,None,None


In [41]:
display(get_number_of_registered_users_for_yesterday(df))

[INFO] [1175540541 - task_user_basic_updates]: Number of users who registered on (2021-12-01) is: 73 [1175540541.get_number_of_registered_users_for:12]


73

In [42]:
df_sf = pd.DataFrame()

df_sf['firstname']         = df['first_name']
# df_sf['Middle_name__c']                = df['middle_name']
df_sf['middlename']        = df['middle_name']
df_sf['lastname']          = df['last_name']
df_sf['Salutation']        = df['title'].apply(lambda s: s + '.' if s != None else s) 

display(df_sf.head())
display(df_sf.tail())

,firstname,middlename,lastname,Salutation
0,hubrepo,None,hubrepo,None
1,nanoHUB,None,support,None
2,Grid,None,Statistics,None
3,NCN,None,NCN,None
4,nanoHUB,None,nanoHUB,None


,firstname,middlename,lastname,Salutation
267057,Arun,None,Jangir,None
267058,Anthony,None,Walton,None
267059,Sourav,None,Mondal,None
267060,Buy,Ambien,Online Overnight,None
267061,chris,None,wallium,None


In [43]:
df_sf['nanoHUB_user_ID__c']            = df['user_id']
df_sf['nanoHUB_username__c']           = df['username']
df_sf['Email']                         = df['email'].fillna('').apply(lambda x: '' if '@invalid' in x else x).apply(lambda x: '' if '@' not in x else x)

# for sendEmail: 0 = opt-out, 1 = receive email. For salesforce HasOptedOutOfEmail, it's exact opposite
df_sf['HasOptedOutOfEmail']            = df['sendEmail'].apply(lambda x: 0 if x==1 else 1)
df_sf['nanoHUB_account_BLOCKED__c']    = df['block'].fillna(0)

# solidify time-related columns from datetime to string
df_sf['nanoHUB_registration_date__c']  = df["registerDate"].fillna('').dt.strftime('%Y-%m-%d')
df_sf['nanoHUB_last_active_date__c']   = df["lastvisitDate"].fillna('').dt.strftime('%Y-%m-%d') 

display(df_sf.head())
display(df_sf.tail())

,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c
0,hubrepo,None,hubrepo,None,998,hubrepo,nkissebe@gmail.com,1,0,2014-11-13,NaN
1,nanoHUB,None,support,None,1683,support,support@nanohub.org,1,0,2008-11-19,2008-11-19
2,Grid,None,Statistics,None,1684,gridstat,gridstat@nanohub.org,1,0,2008-11-18,2020-02-14
3,NCN,None,NCN,None,1685,ncn,ncn@nanohub.org,1,0,2008-11-11,NaN
4,nanoHUB,None,nanoHUB,None,1686,nanohub,apps@nanohub.org,1,0,2014-06-26,NaN


,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c
267057,Arun,None,Jangir,None,347883,-186860,,1,0,2021-12-02,2021-12-02
267058,Anthony,None,Walton,None,347885,-186862,,1,0,2021-12-02,2021-12-02
267059,Sourav,None,Mondal,None,347886,sourav.sm423,sourav.sm423@gmail.com,1,0,2021-12-02,2021-12-02
267060,Buy,Ambien,Online Overnight,None,347888,buyambienonlineovernight,lindseydavid5236@gmail.com,1,0,2021-12-02,2021-12-02
267061,chris,None,wallium,None,347889,-186865,,1,0,2021-12-02,2021-12-02


In [44]:
# sanity checks
display(df_sf[df_sf['nanoHUB_user_ID__c'].isnull()])
display(df_sf[df_sf['nanoHUB_username__c'].isnull()])
display(df_sf[df_sf['Email'].isnull()])
display(df_sf[df_sf['nanoHUB_registration_date__c'].isnull()])
display(df_sf[df_sf['nanoHUB_last_active_date__c'].isnull()])

,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c


,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c


,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c


,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c


,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c
0,hubrepo,None,hubrepo,None,998,hubrepo,nkissebe@gmail.com,1,0,2014-11-13,NaN
3,NCN,None,NCN,None,1685,ncn,ncn@nanohub.org,1,0,2008-11-11,NaN
4,nanoHUB,None,nanoHUB,None,1686,nanohub,apps@nanohub.org,1,0,2014-06-26,NaN
5,Repository,None,Manager,None,1687,repo,repoman@nanohub.org,1,0,2007-11-13,NaN
6,Violin,None,None,None,1688,violin,dxu@cs.purdue.edu,1,0,2007-08-30,NaN
...,...,...,...,...,...,...,...,...,...,...,...
267049,Uniswap,None,Exchange,None,347874,uniexchange,exchangeuniswap@gmail.com,1,0,2021-12-02,NaN
267050,Uniswap,None,Exchange,None,347874,uniexchange,exchangeuniswap@gmail.com,1,0,2021-12-02,NaN
267052,Leo,None,Tsui,None,347877,hcltsui,drleotsui@gmail.com,1,0,2021-12-02,NaN
267053,Leo,None,Tsui,None,347877,hcltsui,drleotsui@gmail.com,1,0,2021-12-02,NaN


In [45]:
display(get_number_of_registered_users_for_yesterday(df_sf, 'nanoHUB_registration_date__c'))

[INFO] [1175540541 - task_user_basic_updates]: Number of users who registered on (2021-12-01) is: 73 [1175540541.get_number_of_registered_users_for:12]


73

In [46]:
# Tableau detailed view
df_sf['Detailed_user_timeline_to_Tableau__c'] = df_sf['Email'].apply(lambda x: 'https://tableauqa.itap.purdue.edu/views/profile/ProfileTimeline?Id%20Email='+x+'\
&:iframeSizedToWindow=true&:embed=y&:showAppBanner=false\
&:display_count=no&:showVizHome=no#6' if x != '' else '')

# nanoHUB user profile
df_sf['nanoHUB_user_page__c'] = df_sf['nanoHUB_user_ID__c'].apply(lambda x: 'https://nanohub.org/members/%d'%x if x != '' else '')
df_sf['Organization__c'] = df['organization'].fillna('')

df_sf['ORCID__c'] = df['orcid'].fillna('')

# derived information
df_sf['Organization_email_derived__c'] = df['Salesforce_org_ID'].fillna(' ')

sf_original_fields = df_sf.columns

# display
display(df_sf.head())
display(df_sf.tail())


,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,Organization__c,ORCID__c,Organization_email_derived__c
0,hubrepo,None,hubrepo,None,998,hubrepo,nkissebe@gmail.com,1,0,2014-11-13,NaN,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/998,none,,
1,nanoHUB,None,support,None,1683,support,support@nanohub.org,1,0,2008-11-19,2008-11-19,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/1683,purdue university,,a0r5w00000V42cCAAR
2,Grid,None,Statistics,None,1684,gridstat,gridstat@nanohub.org,1,0,2008-11-18,2020-02-14,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/1684,purdue university,,a0r5w00000V42cCAAR
3,NCN,None,NCN,None,1685,ncn,ncn@nanohub.org,1,0,2008-11-11,NaN,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/1685,purdue university,,a0r5w00000V42cCAAR
4,nanoHUB,None,nanoHUB,None,1686,nanohub,apps@nanohub.org,1,0,2014-06-26,NaN,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/1686,purdue university,,a0r5w00000V42cCAAR


,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,Organization__c,ORCID__c,Organization_email_derived__c
267057,Arun,None,Jangir,None,347883,-186860,,1,0,2021-12-02,2021-12-02,,https://nanohub.org/members/347883,none,,
267058,Anthony,None,Walton,None,347885,-186862,,1,0,2021-12-02,2021-12-02,,https://nanohub.org/members/347885,none,,
267059,Sourav,None,Mondal,None,347886,sourav.sm423,sourav.sm423@gmail.com,1,0,2021-12-02,2021-12-02,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/347886,maulana abul kalam azad university of technology,,
267060,Buy,Ambien,Online Overnight,None,347888,buyambienonlineovernight,lindseydavid5236@gmail.com,1,0,2021-12-02,2021-12-02,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/347888,"university of california, los angeles",,a0r5w00000V42okAAB
267061,chris,None,wallium,None,347889,-186865,,1,0,2021-12-02,2021-12-02,,https://nanohub.org/members/347889,none,,


In [47]:
display(get_number_of_registered_users_for_yesterday(df_sf, 'nanoHUB_registration_date__c'))

[INFO] [1175540541 - task_user_basic_updates]: Number of users who registered on (2021-12-01) is: 73 [1175540541.get_number_of_registered_users_for:12]


73

## To Salesforce Sales Cloud CRM

In [48]:

db_s.object_id = object_id
db_s.external_id = external_id

In [49]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000bmmmqAAA
hello
[Success] CSV upload successful. Job ID = 7505w00000bmmmqAAA
[Success] Closing job successful. Job ID = 7505w00000bmmmqAAA


In [50]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000bmmmqAAA',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-12-02T15:12:02.000+0000',
 'systemModstamp': '2021-12-02T15:12:59.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [51]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


In [52]:
display(get_number_of_registered_users_for_yesterday(df_sf, 'nanoHUB_registration_date__c'))

[INFO] [1175540541 - task_user_basic_updates]: Number of users who registered on (2021-12-01) is: 73 [1175540541.get_number_of_registered_users_for:12]


73

In [53]:
display(df_sf.loc[df_sf['nanoHUB_user_ID__c'] == 288227])

,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,Organization__c,ORCID__c,Organization_email_derived__c
218249,Anurag,Kumar,Tiwari,Dr,288227,tiwaria,tiwaria@nitj.ac.in,0,0,2020-05-18,2021-03-17,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/288227,nitjalandhar,,


,firstname,middlename,lastname,Salutation,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,Organization__c,ORCID__c,Organization_email_derived__c
218249,Anurag,Kumar,Tiwari,Dr.,288227,tiwaria,tiwaria@nitj.ac.in,0,0,2020-05-18,2021-03-17,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/288227,nitjalandhar,,
